In [5]:
import textstat
import pandas as pd
import spacy
import subprocess
subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


CompletedProcess(args=['python', '-m', 'spacy', 'download', 'en_core_web_sm'], returncode=0)

In [6]:
# Load English SpaCy model
nlp = spacy.load("en_core_web_sm") #In terminal: python -m spacy download en_core_web_sm

# Download concreteness lexicon
# Direct URL to the CSV file (latest version, Conc.M = concreteness mean)
url = "hf://datasets/StephanAkkerman/MRC-psycholinguistic-database/mrc_psycholinguistic_database.csv"
# Load into pandas
lexicon = pd.read_csv(url)[['Word','Concreteness', 'Imageability']]
lexicon["Word"] = lexicon["Word"].str.lower()
lexicon.head()

,Word,Concreteness,Imageability
0,&arry,0,0
1,&cello,0,0
2,&d,0,0
3,&em,0,0
4,&flu,0,0


In [7]:
# Define the function

def analyze_text(text):
    doc = nlp(text)
    words = [token.text.lower() for token in doc if token.is_alpha]

    # Readability (Flesch Reading Ease)
    readability = textstat.flesch_reading_ease(text)

    # Imageability & Concreteness (averages)
    imageability_scores = [
        lexicon.loc[lexicon['Word'] == word, 'Imageability'].values[0]
        for word in words if word in lexicon['Word'].values and 'Imageability' in lexicon.columns
    ]
    concreteness_scores = [
        lexicon.loc[lexicon['Word'] == word, 'Concreteness'].values[0]
        for word in words if word in lexicon['Word'].values and 'Concreteness' in lexicon.columns
    ]

    avg_imageability = sum(imageability_scores) / len(imageability_scores) if imageability_scores else None
    avg_concreteness = sum(concreteness_scores) / len(concreteness_scores) if concreteness_scores else None

    # Syntactic simplicity (average number of words per sentence)
    avg_sentence_length = sum(len(sent) for sent in doc.sents) / len(list(doc.sents))

    return {
        "Readability": round(readability, 2),
        "Imageability": round(avg_imageability, 2) if avg_imageability else "N/A",
        "Concreteness": round(avg_concreteness, 2) if avg_concreteness else "N/A",
        "Syntactic simplicity (avg sentence length)": round(avg_sentence_length, 2)
    }


## Classic

In [8]:
# Exemple d'utilisation
poems = pd.read_excel("/home/onyxia/work/Phoetry/generated_poems/poem_classic_.xlsx")

poems.head()

,theme,text
0,moon,Then we see the moon shining on her face. It's...
1,leaf,I only I could have the leaf. But it's not lik...
2,flower,For I am the flower of life. And that is what ...
3,tree,Then we see the tree fall down to its own litt...
4,sun,I only I could have the sun. When it was dark ...


In [9]:
poems["Readability"]=poems["text"].apply(lambda x: analyze_text(x)["Readability"])
poems["Imageability"]=poems["text"].apply(lambda x: analyze_text(x)["Imageability"])
poems["Concreteness"]=poems["text"].apply(lambda x: analyze_text(x)["Concreteness"])
poems["Syntactic simplicity"]=poems["text"].apply(lambda x: analyze_text(x)["Syntactic simplicity (avg sentence length)"])
poems


,theme,text,Readability,Imageability,Concreteness,Syntactic simplicity
0,moon,Then we see the moon shining on her face. It's...,101.29,329.57,298.64,14.75
1,leaf,I only I could have the leaf. But it's not lik...,99.57,305.75,274.91,21.00
2,flower,For I am the flower of life. And that is what ...,85.73,279.95,251.09,20.33
3,tree,Then we see the tree fall down to its own litt...,89.08,293.44,261.56,18.00
4,sun,I only I could have the sun. When it was dark ...,99.57,288.22,267.02,16.50
5,sunset,For I am the sunset; for it is my light that s...,85.02,300.74,271.41,21.75
6,waterfall,Then we see the waterfall that is at first. It...,69.75,263.41,232.03,28.33
7,butterfly,I only I could have the butterfly. When it was...,90.29,304.07,265.86,17.40
8,bird,Then we see the bird come out of its shell. Th...,90.29,286.56,249.60,17.40
9,dog,For I am the dog who is to be eaten by wolves....,84.00,294.23,245.16,23.00


In [10]:
 #normalize scores
poems_normalized=poems.copy()
cols_to_normalize = ['Readability', 'Imageability', 'Concreteness', 'Syntactic simplicity']

for col in cols_to_normalize:
    min_val = poems_normalized[col].min()
    max_val = poems_normalized[col].max()
    poems_normalized[col] = (poems_normalized[col] - min_val) / (max_val - min_val)
    poems_normalized[col]=poems_normalized[col].apply(lambda x: round(x,2))
poems_normalized

,theme,text,Readability,Imageability,Concreteness,Syntactic simplicity
0,moon,Then we see the moon shining on her face. It's...,1.00,1.00,1.00,0.01
1,leaf,I only I could have the leaf. But it's not lik...,0.95,0.65,0.64,0.46
2,flower,For I am the flower of life. And that is what ...,0.58,0.28,0.29,0.41
3,tree,Then we see the tree fall down to its own litt...,0.67,0.47,0.44,0.24
4,sun,I only I could have the sun. When it was dark ...,0.95,0.40,0.53,0.13
5,sunset,For I am the sunset; for it is my light that s...,0.56,0.58,0.59,0.52
6,waterfall,Then we see the waterfall that is at first. It...,0.15,0.03,0.00,1.00
7,butterfly,I only I could have the butterfly. When it was...,0.70,0.63,0.51,0.20
8,bird,Then we see the bird come out of its shell. Th...,0.70,0.37,0.26,0.20
9,dog,For I am the dog who is to be eaten by wolves....,0.53,0.48,0.20,0.61


In [11]:
poems_normalized.describe()

,Readability,Imageability,Concreteness,Syntactic simplicity
count,13.000000,13.000000,13.000000,13.000000
mean,0.617692,0.475385,0.441538,0.395385
std,0.290980,0.289759,0.281509,0.315479
min,0.000000,0.000000,0.000000,0.000000
25%,0.530000,0.370000,0.260000,0.200000
50%,0.670000,0.470000,0.440000,0.410000
75%,0.720000,0.630000,0.590000,0.520000
max,1.000000,1.000000,1.000000,1.000000


In [12]:
poems.to_excel("Scores_poems.xlsx")

## Haiku

In [13]:
# comparaison
haiku = pd.read_excel("/home/onyxia/work/Phoetry/generated_poems/poem_haiku_.xlsx")
haiku.head()

,theme,text
0,moon,The space between?\nMy fingers and lips. I ho...
1,leaf,The smell of grasshoppers. ppening. Grass In t...
2,flower,The sound of her hand. Sings in the wind. Jasm...
3,tree,A leaf falls. From the tree's branch. Leaf by...
4,sun,The shape of the moon. Sighs me outstretched....


In [14]:
haiku["Readability"] = haiku["text"].apply(lambda x: analyze_text(x)["Readability"])
haiku["Imageability"] = haiku["text"].apply(lambda x: analyze_text(x)["Imageability"])
haiku["Concreteness"] = haiku["text"].apply(lambda x: analyze_text(x)["Concreteness"])
haiku["Syntactic simplicity"] = haiku["text"].apply(lambda x: analyze_text(x)["Syntactic simplicity (avg sentence length)"])
haiku

,theme,text,Readability,Imageability,Concreteness,Syntactic simplicity
0,moon,The space between?\nMy fingers and lips. I ho...,92.80,244.17,226.83,4.50
1,leaf,The smell of grasshoppers. ppening. Grass In t...,83.83,335.38,322.88,3.00
2,flower,The sound of her hand. Sings in the wind. Jasm...,108.19,306.30,300.90,3.75
3,tree,A leaf falls. From the tree's branch. Leaf by...,117.87,393.00,379.85,4.75
4,sun,The shape of the moon. Sighs me outstretched....,101.56,233.80,237.80,5.00
5,sunset,A water lily's cry. lessening. whispers of My...,66.91,336.56,321.00,4.67
6,waterfall,The slow rise and fall of waterdrops. Waterdr...,83.62,304.92,263.08,5.67
7,butterfly,A single leaf. Pours out of the sky. Dreaming...,91.27,277.64,216.55,3.75
8,bird,The call of a hawk. Whisper Between us. Osprey's,92.29,294.78,279.67,4.33
9,dog,The scent of lilacs. Dapples in the air. Over...,83.32,184.00,191.20,3.50


In [15]:
#normalize scores
haiku_normalized=haiku.copy()
cols_to_normalize = ['Readability', 'Imageability', 'Concreteness', 'Syntactic simplicity']

for col in cols_to_normalize:
    min_val = haiku_normalized[col].min()
    max_val = haiku_normalized[col].max()
    haiku_normalized[col] = (haiku_normalized[col] - min_val) / (max_val - min_val)
    haiku_normalized[col]=haiku_normalized[col].apply(lambda x: round(x,2))
haiku_normalized


,theme,text,Readability,Imageability,Concreteness,Syntactic simplicity
0,moon,The space between?\nMy fingers and lips. I ho...,0.51,0.29,0.19,0.56
1,leaf,The smell of grasshoppers. ppening. Grass In t...,0.33,0.72,0.70,0.00
2,flower,The sound of her hand. Sings in the wind. Jasm...,0.81,0.59,0.58,0.28
3,tree,A leaf falls. From the tree's branch. Leaf by...,1.00,1.00,1.00,0.66
4,sun,The shape of the moon. Sighs me outstretched....,0.68,0.24,0.25,0.75
5,sunset,A water lily's cry. lessening. whispers of My...,0.00,0.73,0.69,0.63
6,waterfall,The slow rise and fall of waterdrops. Waterdr...,0.33,0.58,0.38,1.00
7,butterfly,A single leaf. Pours out of the sky. Dreaming...,0.48,0.45,0.13,0.28
8,bird,The call of a hawk. Whisper Between us. Osprey's,0.50,0.53,0.47,0.50
9,dog,The scent of lilacs. Dapples in the air. Over...,0.32,0.00,0.00,0.19


In [16]:
haiku_normalized.describe()

,Readability,Imageability,Concreteness,Syntactic simplicity
count,13.000000,13.000000,13.000000,13.000000
mean,0.479231,0.492308,0.422308,0.452308
std,0.265156,0.255707,0.277794,0.269201
min,0.000000,0.000000,0.000000,0.000000
25%,0.330000,0.340000,0.240000,0.280000
50%,0.490000,0.530000,0.380000,0.470000
75%,0.620000,0.590000,0.580000,0.630000
max,1.000000,1.000000,1.000000,1.000000


In [17]:
haiku.to_excel("Scores_haiku.xlsx")